In [119]:
import random
import json
import pandas as pd

class WeightedRandomizer:
    def __init__ (self, weights):
        self.__max = .0
        self.__weights = []
        for value, weight in weights.items ():
            self.__max += weight
            self.__weights.append ( (self.__max, value) )

    def random (self):
        r = random.random () * self.__max
        for ceil, value in self.__weights:
            if ceil > r: return value
            
w = { "Legendary": 0.75, "Epic": 3.76, "Rare": 16.9, "Common": 78.59}
wr = WeightedRandomizer(w)

def pick_rarities():
    return [wr.random() if i not in [0,14,29] else "Rare" for i in range(30)]

with open("AllSets.enUS.json", 'r') as fp:
    all = json.load(fp)
    
# don't load cards from these sets, they aren't used in Arena
excluded_sets = ["Hero Skins", "Missions", "System", "Credits", "Debug", "Promotion", "Tavern Brawl", "Reward"]

arena_sets = [s for s in all.keys() if s not in excluded_sets]

#since many fields are missing, we first need to build a set of valid columns names
column_names = set()
for set_name in arena_sets:
    for card in all[set_name]:
        column_names = column_names.union(set(card.keys()))

column_names.add("set")
column_names.add("draft_weight")

card_values = dict()
for col in column_names:
    card_values[col] = []
    
weighted_set = u'League of Explorers'
for set_name in arena_sets:
    for card in all[set_name]:
        for col in column_names:
            if col == "set":
                card_values["set"].append(set_name)
            elif col == "draft_weight":
                card_values["draft_weight"].append(1.0 if set_name != weighted_set else 1.25)
            else:
                card_values[col].append(card.get(col, None))

cards_df = pd.DataFrame(data=card_values)

# "Free" cards should be "Common"
cards_df[cards_df.rarity == "Free"] = "Common"

# TODO: Remove wing 3 and 4 LOE cards
cards_df = cards_df.set_index("id")

# drop any non-real cards
cards_df = cards_df.dropna(subset=["cost"])
cards_df = cards_df[cards_df.collectible == True]

player_classes = [u'Mage', u'Paladin', u'Warlock', u'Warrior', u'Hunter', u'Priest', u'Shaman', u'Druid', u'Rogue']
draftables = dict()

for c in player_classes:
    # Class cards get a 100% weight bump
    class_cards = cards_df[cards_df.playerClass == c].copy()
    class_cards.draft_weight += 1
    
    # and then add the neutral cards
    class_cards = class_cards.append(cards_df[cards_df.playerClass.isnull()])
    draftables[c] = class_cards
    

def draft_options(playerClass, rarity):
    possible = draftables[playerClass]
    possible = possible[possible.rarity == rarity]
    picks = []
    for i in range(3):
        picks.append(possible.sample(weights="draft_weight").iloc[0])
    return picks

def draft_picks(playerClass):
    rarities = pick_rarities()
    picks = []
    for rarity in rarities:
        picks.append(draft_options(playerClass, rarity))
    return picks

    
cards_df.describe()

,artist,attack,collectible,cost,draft_weight,durability,elite,faction,flavor,health,...,howToGetGold,inPlayText,mechanics,name,playerClass,race,rarity,set,text,type
count,680,493,680,680,680,21,91,264,679,472,...,135,25,393,680,388,155,680,680,665,680
unique,226,12,1,13,2,6,2,3,679,12,...,102,24,41,680,9,7,4,7,617,3
top,Alex Horley Orlandelli,2,True,3,1,2,True,Neutral,"If she had studied harder, she would have been...",4,...,Unlocked at Level 49.,Summoning,[Battlecry],Ice Block,Hunter,Mech,Common,Classic,<b>Taunt</b>,Minion
freq,38,110,680,138,635,11,89,213,1,95,...,7,2,152,1,44,48,315,245,14,472


In [126]:
options = draft_picks("Warrior") #ArenaWarriorsMatter
[ {row[0].rarity: [c["name"] for c in row] } for row in options]

[{u'Rare': [u'Young Priestess', u'Commanding Shout', u'Eerie Statue']},
 {u'Common': [u'Blackwing Corruptor', u"Kor'kron Elite", u'Frigid Snobold']},
 {u'Epic': [u'Crush', u'Twilight Guardian', u'Sea Reaver']},
 {u'Common': [u'Force-Tank MAX', u'Wisp', u'Maiden of the Lake']},
 {u'Rare': [u'Mortal Strike', u'Arcane Golem', u'Siege Engine']},
 {u'Common': [u'Shieldbearer', u'Obsidian Destroyer', u'Whirlwind']},
 {u'Epic': [u'Sea Reaver', u'Kodorider', u'Shield Slam']},
 {u'Common': [u'Silvermoon Guardian',
   u'Darkscale Healer',
   u'North Sea Kraken']},
 {u'Common': [u'Argent Squire', u'Blackwing Corruptor', u'Faerie Dragon']},
 {u'Common': [u'Ogre Warmaul', u'Razorfen Hunter', u'Bloodsail Raider']},
 {u'Common': [u'Captured Jormungar', u'Bash', u'Rampage']},
 {u'Common': [u"Death's Bite", u'Burly Rockjaw Trogg', u'Slam']},
 {u'Common': [u'Stormpike Commando', u'Frostwolf Grunt', u'Pit Fighter']},
 {u'Common': [u'Burly Rockjaw Trogg',
   u'Arathi Weaponsmith',
   u'Flesheating Ghoul']